# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet (`outputSet`) that lie on the convex hull.

In [1]:
def jarvismarch(inputSet):      
    '''
    Returns the list of inputSet that lie on the convex hull (jarvis march algorithm)
            Parameters:
                    inputSet (list): a list of 2D inputSet

            Returns:
                    outputSet (list): a list of 2D inputSet
    '''

    leftMostPoint = findLeftMostPoint(inputSet)
    outputSet = []
        
    origin = inputSet[leftMostPoint]
    outputSet.append(origin)
        
    p = leftMostPoint
        
    while True:
        q = (p + 1) % len(inputSet)
        for i in range(len(inputSet)):
                if isBetterOrientation(inputSet[p], inputSet[q], inputSet[i]):
                        q = i
        p = q
        if p == leftMostPoint:
                break
                
        outputSet.append(inputSet[p])

    return outputSet

Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [2]:
# used for copying over array elements
def subArrayCopy(arr, subArr, start, end):
    subArrayIndex = 0
    for i in range(start, end + 1):
        subArr[subArrayIndex] = arr[i]
        subArrayIndex += 1

def merge(arr, leftP, rightP):
    middleI = (rightP + leftP) // 2

    leftArray = [Point(0, 0)] * (middleI + 1 - leftP)
    rightArray = [Point(0, 0)] * (rightP - middleI)

    subArrayCopy(arr, leftArray, leftP, middleI)
    subArrayCopy(arr, rightArray, middleI + 1, rightP)

    leftArrayI = 0
    rightArrayI = 0
    arrPointer = leftP

    while leftArrayI < len(leftArray) and rightArrayI < len(rightArray):
        leftPoint = leftArray[leftArrayI]
        rightPoint = rightArray[rightArrayI]

        # orders inputSet by orientation, relative to leftmostpoint
        if isBetterOrientation(arr[0], leftPoint, rightPoint):
            arr[arrPointer] = leftArray[leftArrayI]
            leftArrayI += 1
        else:
            arr[arrPointer] = rightArray[rightArrayI]
            rightArrayI += 1

        arrPointer += 1

    while leftArrayI < len(leftArray):
        arr[arrPointer] = leftArray[leftArrayI]
        leftArrayI += 1
        arrPointer += 1

    while rightArrayI < len(rightArray):
        arr[arrPointer] = rightArray[rightArrayI]
        rightArrayI += 1
        arrPointer += 1

def mergeSort(arr, leftP, rightP):
    if leftP < rightP:
        middleI = (rightP + leftP) // 2

        mergeSort(arr, leftP, middleI)
        mergeSort(arr, middleI + 1, rightP)

        merge(arr, leftP, rightP)

def grahamScan(inputSet):
    bottomLeftmostPoint = findLeftMostPoint(inputSet)
    
    #positions leftmostpoint as the first point
    #and sorts the rest of the points by orientation
    inputSet[bottomLeftmostPoint], inputSet[0] = inputSet[0], inputSet[bottomLeftmostPoint]
    mergeSort(inputSet, 1, len(inputSet) - 1)

    convexHull = []
    convexHull.append(inputSet[0])
    convexHull.append(inputSet[1])

    for i in range(2, len(inputSet)):

        # keeps popping inputSet off stack until back to anticlockwise convex hull
        while len(convexHull) > 1 and not isBetterOrientation(convexHull[-2], convexHull[-1], inputSet[i]):
            convexHull.pop()
        convexHull.append(inputSet[i])

    return convexHull



Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [3]:

def jarvisMarchModified(hulls):

    # finds the leftmost point from all sub convex hulls
    # in sorting at grahams, each leftmost point is put at 0th index
    bottomLeftPoints = [hull[0] for hull in hulls]
    leftMostHullI = findLeftMostPoint(bottomLeftPoints)


    p = hulls[leftMostHullI][0]
    q = hulls[leftMostHullI][1]
    outputSet = [p]


    # searches each sub convex hull for a more anticlockwise point
    while True:
        for hull in hulls:
            for r in hull:    
                if isBetterOrientation(p, q, r):
                    q = r
        p = q
        if p == outputSet[0]:
            break

        outputSet.append(p)

    return outputSet

def createSubsets(inputSet):
    # size of convex hull approximated at root of total number of points
    m = int(math.sqrt(len(inputSet)))
    while len(inputSet) % m < 3:
        m +=1
    
    subsets = []
    
    for i in range(0, len(inputSet), m):
        subset = inputSet[i: i + m]
        subsets.append(subset)

    return subsets


def chenScan(inputSet):
    # portions points into subsets
    subsets = createSubsets(inputSet)

    # uses grahams to form sub convex hulls
    subConvexHulls = [grahamScan(subset) for subset in subsets]

    # forms final convex hull using jarvis march
    outputSet = jarvisMarchModified(subConvexHulls)
    return outputSet


''' Alternative Chens, faster only for really really large numbers of points 



def createSubsets(inputSet, m):
    subsets = []
    for i in range(0, len(inputSet), m):
        subset = inputSet[i: i + m]
        if len(subset) < 3:
            return []
        subsets.append(subset)

    return subsets

# attempts to solve for a convex hull of size m
# if it fails it then tries for a larger size m (previous m squared each failure)
def chensScan(inputSet):
    t = 0
    outputSet = []

    while not outputSet:
        t += 1
        m = 2 ** 2 ** t

    
        # portions points into subsets
        subsets = createSubsets(inputSet, m)

        #invalid number of subsets, move onto next m
        if not subsets:
            continue
        
        # forms sub convex hulls using grahams
        # then creates a final convex hull using jarvis
        hulls = [grahamScan(subset) for subset in subsets]
        outputSet = jarvisMarchModified(hulls, m)

    return outputSet

'''


Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [4]:
import math
import random
import matplotlib.pyplot


#auxiliary components
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def findLeftMostPoint(inputSet):
        leftMostIndex = 0
        for i in range(1, len(inputSet)):
                currentLeftMost = inputSet[leftMostIndex]
                currentPoint = inputSet[i]
                
                # Update to new left most point
                if currentPoint.x < currentLeftMost.x:
                        leftMostIndex = i
                
                # If multiple inputSet have same x coord, prioritise largest y
                elif currentPoint.x == currentLeftMost.x:
                        if currentPoint.y > currentLeftMost.y:
                                leftMostIndex = i
                        
        return leftMostIndex
    
def calcDistance(p, q):
        return abs(p.x - q.x) + abs(p.y - q.y)


def isBetterOrientation(p, q, r):
        crossProduct = (q.x - p.x) * (r.y - p.y) - (q.y - p.y) * (r.x - p.x)
        
        # Counterclockwise - accept
        if crossProduct < 0:
                return True
        # Collinear - only accept if further away
        elif crossProduct == 0:
                if calcDistance(p, r) > calcDistance(p, q):
                        return True
        
        return False



''' not sure how to order , apologies for the mess '''


class TestDataGenerator():
    """
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass


Use the cell below to implement the requested **experimental framework** API.

In [5]:
import timeit
import matplotlib

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass

Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [6]:
# ADD YOUR TEST CODE HERE 



